In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Scrape Data 

In [2]:
starting_week = 19
current_week = 23

In [3]:
positions = ['QB', 'WR', 'RB', 'TE', 'K']
weeks = [week for week in range(starting_week,current_week +1)]

file_list = []

for position in positions:
    for week in weeks:
        url = f'https://www.cbssports.com/nfl/stats/leaders/live/{position}/{week}/'
        print(url)
        res = requests.get(url)
        print(res)
        output = open(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls', 'wb')
        output.write(res.content)
        file_list.append(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        output.close()
        data = pd.read_html(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        df = data[0]
        # df = df.droplevel(0, axis=1)
        print((position, week), df.shape)

https://www.cbssports.com/nfl/stats/leaders/live/QB/19/
<Response [200]>
('QB', 19) (13, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/20/
<Response [200]>
('QB', 20) (8, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/21/
<Response [200]>
('QB', 21) (3, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/22/
<Response [200]>
('QB', 22) (6, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/23/
<Response [200]>
('QB', 23) (2, 11)
https://www.cbssports.com/nfl/stats/leaders/live/WR/19/
<Response [200]>
('WR', 19) (56, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/20/
<Response [200]>
('WR', 20) (39, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/21/
<Response [200]>
('WR', 21) (14, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/22/
<Response [200]>
('WR', 22) (9, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/23/
<Response [200]>
('WR', 23) (10, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/19/
<Response [200]>
('

In [4]:
# get df started with the 1st file './nfl_data/week-19_QB.xls'.  This way we have something to merge to.

if 'QB' in file_list[0]:
    df_qb = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_QB.xls')
    df_qb = pd.DataFrame(df_qb[0])
    df_qb = df_qb.droplevel(0, axis=1)
    df_qb['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
    df_qb['Pos'] = [i.split()[-1] for i in df_qb['Player  Player on team']]
    df_qb.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
    df_qb['Player'] = df_qb['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])

for file in file_list[1:]:
    if 'QB' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_qb, df]
        df_qb = pd.concat([df_qb.squeeze() for df_qb in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_qb.shape}')

./nfl_data/weekly/cbs_weekly/week-20_QB.xls
./nfl_data/weekly/cbs_weekly/week-21_QB.xls
./nfl_data/weekly/cbs_weekly/week-22_QB.xls
./nfl_data/weekly/cbs_weekly/week-23_QB.xls
The shape of the updated df is (32, 13)


In [5]:
df_qb.tail()

,Player,Game,FPTS Fantasy Points,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
27,Kirk Cousins,AFC 41 - NFC 35,4,6,10,66,1,1,—,—,—,22,QB
28,Russell Wilson,AFC 41 - NFC 35,—,9,17,77,0,2,—,—,—,22,QB
29,Patrick Mahomes,AFC 41 - NFC 35,—,5,10,53,0,1,—,—,—,22,QB
30,Matthew Stafford,LAR 23 - CIN 20,25,26,40,283,3,2,3,6,0,23,QB
31,Joe Burrow,LAR 23 - CIN 20,16,22,33,263,1,0,2,3,0,23,QB


In [6]:
def impute_special_char(df, char):
    df.replace(char, 0, inplace = True) # get rid of the dashes, impute with 0
    
impute_special_char(df_qb, '—')

In [7]:
def change_col_types(df):
    numcols_to_change = df.columns
    numcols_to_change2 = []
    for col in numcols_to_change:
        try:
            df[col] = df[col].astype(int)
            print('success!')
        except:
            numcols_to_change2.append(col)
            print(f'need to clean column: {col}')
            
change_col_types(df_qb)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [8]:
# get df started with the 1st file './nfl_data/week-19_RB.xls'.  This way we have something to merge to.

df_rb = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_RB.xls')
df_rb = pd.DataFrame(df_rb[0])
df_rb = df_rb.droplevel(0, axis=1)
df_rb['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_rb['Pos'] = [i.split()[-1] for i in df_rb['Player  Player on team']]
df_rb.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_rb['Player'] = df_rb['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])

for file in file_list[1:]:
    if 'RB' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_rb, df]
        df_rb = pd.concat([df_rb.squeeze() for df_rb in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_rb.shape}')

./nfl_data/weekly/cbs_weekly/week-19_RB.xls
./nfl_data/weekly/cbs_weekly/week-20_RB.xls
./nfl_data/weekly/cbs_weekly/week-21_RB.xls
./nfl_data/weekly/cbs_weekly/week-22_RB.xls
./nfl_data/weekly/cbs_weekly/week-23_RB.xls
The shape of the updated df is (112, 12)


In [9]:
impute_special_char(df_rb, '—')
change_col_types(df_rb)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [10]:
df_wr = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_WR.xls')
df_wr = pd.DataFrame(df_wr[0])
df_wr = df_wr.droplevel(0, axis=1)
df_wr['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_wr['Pos'] = [i.split()[-1] for i in df_wr['Player  Player on team']]
df_wr.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_wr['Player'] = df_wr['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])


for file in file_list:
    if 'WR' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_wr, df]
        df_wr = pd.concat([df_wr.squeeze() for df_wr in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_wr.shape}')

./nfl_data/weekly/cbs_weekly/week-19_WR.xls
./nfl_data/weekly/cbs_weekly/week-20_WR.xls
./nfl_data/weekly/cbs_weekly/week-21_WR.xls
./nfl_data/weekly/cbs_weekly/week-22_WR.xls
./nfl_data/weekly/cbs_weekly/week-23_WR.xls
The shape of the updated df is (184, 12)


In [11]:
impute_special_char(df_wr, '—')
change_col_types(df_wr)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [12]:
df_te = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_TE.xls')
df_te = pd.DataFrame(df_te[0])
df_te = df_te.droplevel(0, axis=1)
df_te['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_te['Pos'] = [i.split()[-1] for i in df_te['Player  Player on team']]
df_te.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_te['Player'] = df_te['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])

for file in file_list:
    if 'TE' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_te, df]
        df_te = pd.concat([df_te.squeeze() for df_te in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_te.shape}')

./nfl_data/weekly/cbs_weekly/week-19_TE.xls
./nfl_data/weekly/cbs_weekly/week-20_TE.xls
./nfl_data/weekly/cbs_weekly/week-21_TE.xls
./nfl_data/weekly/cbs_weekly/week-22_TE.xls
./nfl_data/weekly/cbs_weekly/week-23_TE.xls
The shape of the updated df is (70, 12)


In [13]:
impute_special_char(df_te, '—')
change_col_types(df_te)

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [14]:
df_k = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_K.xls')
df_k = pd.DataFrame(df_k[0])
#df_k = df_k.droplevel(0, axis=1)
df_k['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_k['Pos'] = [i.split()[-1] for i in df_k['Player  Player on team']]
df_k.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_k['Player'] = df_k['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])

for file in file_list:
    if 'K' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        #df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_k, df]
        df_k = pd.concat([df_k.squeeze() for df_k in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_k.shape}')

./nfl_data/weekly/cbs_weekly/week-19_K.xls
./nfl_data/weekly/cbs_weekly/week-20_K.xls
./nfl_data/weekly/cbs_weekly/week-21_K.xls
./nfl_data/weekly/cbs_weekly/week-22_K.xls
./nfl_data/weekly/cbs_weekly/week-23_K.xls
The shape of the updated df is (39, 8)


In [15]:
impute_special_char(df_k, '—')
change_col_types(df_k)
# FGM-A and XMP-A do not need to be cleaned, have dashes.

need to clean column: Player
need to clean column: Game
success!
need to clean column: FGM-A  Field Goals Made - Field Goals Attempted
success!
need to clean column: XPM-A  Extra Points Made - Extra Points Attempted
success!
need to clean column: Pos


## Merge all stats together in one df, `df_stats`

In [16]:
print(f'df_rb shape is {df_rb.shape}')
print(f'df_wr shape is {df_wr.shape}')
print(f'df_qb shape is {df_qb.shape}')
print(f'df_te shape is {df_te.shape}')
print(f'df_k shape is {df_k.shape}')

df_rb shape is (112, 12)
df_wr shape is (184, 12)
df_qb shape is (32, 13)
df_te shape is (70, 12)
df_k shape is (39, 8)


In [17]:
df_1 = pd.merge(df_rb, df_k, how = 'outer')

df_2 = pd.merge(df_1, df_qb, how = 'outer')

df_3 = pd.merge(df_2, df_wr, how = 'outer')

df_stats = pd.merge(df_3, df_te, how = 'outer')

In [18]:
df_stats.fillna(0, inplace = True)

In [19]:
df_stats.head()

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown
0,Devin Singletary,BUF 47 - NE 17,21,16.0,81.0,2.0,3.0,13.0,4.0,0.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,Devin Singletary,BUF 47 - NE 17,21,16.0,81.0,2.0,3.0,13.0,4.0,0.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,Jerick McKinnon,KC 42 - PIT 21,20,12.0,61.0,0.0,6.0,81.0,6.0,1.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,Jerick McKinnon,KC 42 - PIT 21,20,12.0,61.0,0.0,6.0,81.0,6.0,1.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,Elijah Mitchell,SF 23 - DAL 17,15,27.0,96.0,1.0,1.0,-11.0,2.0,0.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_stats.shape

(437, 20)

In [21]:
df_stats.drop_duplicates(inplace = True)

In [22]:
df_stats.shape

(312, 20)

### Scrape football reference, to get player information

In [23]:
url = 'https://www.pro-football-reference.com/years/2021/rushing.htm'
resp = requests.get(url)
resp

output = open('./nfl_data/2021_rushing_stats.xls', 'wb')
output.write(resp.content)
output.close()

In [24]:
position = ['rushing', 'passing', 'receiving']

for element in position:
    url = f'https://www.pro-football-reference.com/years/2021/{element}.htm'
    resp = requests.get(url)
    
    output = open(f'./nfl_data/2021_{element}_stats.xls', 'wb')
    output.write(resp.content)
    output.close()

In [25]:
pi_wr = pd.read_html('./nfl_data/2021_receiving_stats.xls')
pi_wr = pd.DataFrame(pi_wr[0]) # Saves df var to dataframe
#df_wr = df_rb.droplevel(0, axis=1) # Removes first level column

pi_qb = pd.read_html('./nfl_data/2021_passing_stats.xls')
pi_qb = pd.DataFrame(pi_qb[0]) # Saves df var to dataframe
#df_qb = df_rb.droplevel(0, axis=1) # Removes first level column

pi_rb = pd.read_html('./nfl_data/2021_rushing_stats.xls')
pi_rb = pd.DataFrame(pi_rb[0]) # Saves df var to dataframe
pi_rb = pi_rb.droplevel(0, axis=1) # Removes first level column

In [26]:
def drop_rows(position):
    for header in position.columns:
        index_list = position.loc[(position[header] == header)].index
        position.drop(labels=index_list, axis=0, inplace = True)
        return
    
drop_rows(pi_rb)
drop_rows(pi_wr)
drop_rows(pi_qb)

In [27]:
pi_wr['Player'] = pi_wr['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))
pi_qb['Player'] = pi_qb['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))
pi_rb['Player'] = pi_rb['Player'].map(lambda x: x.rstrip('_+!*@#$?^'))

In [28]:
dfs = [pi_wr, pi_qb, pi_rb]
player_info = pd.concat([player_info.squeeze() for player_info in dfs], ignore_index=True)

In [29]:
player_info = player_info[['Player', 'Tm']].copy()

In [30]:
player_info.rename(columns = {'Tm':'Team_Name_Abbrev'}, inplace = True)

In [31]:
player_info = player_info.drop_duplicates()

In [32]:
player_info.head(10)

,Player,Team_Name_Abbrev
0,Cooper Kupp,LAR
1,Davante Adams,GNB
2,Tyreek Hill,KAN
3,Justin Jefferson,MIN
4,Mark Andrews,BAL
5,Diontae Johnson,PIT
6,Keenan Allen,LAC
7,Jaylen Waddle,MIA
8,Stefon Diggs,BUF
9,Hunter Renfrow,LVR


In [33]:
player_info_dict = pd.Series(player_info.Team_Name_Abbrev.values,index=player_info.Player).to_dict()
# create dictionary with {player : team abbreviation}

In [34]:
Team_Abbreviations_Dict = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills' : 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GNB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KAN',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NWE',
    'New Orleans Saints': 'NO',
    'NY Giants': 'NYG',
    'NY Jets': 'NYJ',
    'Las Vegas Raiders': 'LVR',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'Los Angeles Chargers': 'LAC',
    'San Francisco 49ers': 'SFO',
    'Seattle Seahawks': 'SEA',
    'Los Angeles Rams': 'LAR',
    'Tampa Bay Buccaneers': 'TAM',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

In [35]:
def get_key(val):
    for key, value in Team_Abbreviations_Dict.items():
         if val == value:
            return key
 
    return f'key does not exist --> {val}'

In [36]:
player_info['Team_Name_Full'] = player_info['Team_Name_Abbrev'].map(get_key)

In [37]:
player_info['Team_Name_Mascot'] = player_info['Team_Name_Full'].map(lambda x: x.split()[-1])

### Merge `player_info` with `df_stats`

In [38]:
df = pd.merge(df_stats, player_info, on = 'Player', how = 'inner')

In [39]:
df.head()

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot
0,Devin Singletary,BUF 47 - NE 17,21,16.0,81.0,2.0,3.0,13.0,4.0,0.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0,BUF,Buffalo Bills,Bills
1,Devin Singletary,KC 42 - BUF 36,14,10.0,26.0,1.0,4.0,25.0,5.0,0.0,20,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0,BUF,Buffalo Bills,Bills
2,Jerick McKinnon,KC 42 - PIT 21,20,12.0,61.0,0.0,6.0,81.0,6.0,1.0,19,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0,KAN,Kansas City Chiefs,Chiefs
3,Jerick McKinnon,KC 42 - BUF 36,12,10.0,24.0,0.0,5.0,54.0,7.0,0.0,20,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0,KAN,Kansas City Chiefs,Chiefs
4,Jerick McKinnon,CIN 27 - KC 24,12,12.0,65.0,0.0,3.0,30.0,4.0,0.0,21,RB,0,0.0,0,0.0,0.0,0.0,0.0,0.0,KAN,Kansas City Chiefs,Chiefs


## Scrape weather

In [40]:
weeks = [f'week-{week}' for week in range(1,17)]

file_list = []

for week in weeks:
    url = f'https://www.nflweather.com/en/week/2021/{week}/'
    print(url)
    res = requests.get(url)
    print(res)
    output = open(f'./nfl_data/weekly/weather/{week}_weather.xls', 'wb')
    output.write(res.content)
    file_list.append(f'./nfl_data/weekly/weather/{week}_weather.xls')
    output.close()
    data = pd.read_html(f'./nfl_data/weekly/weather/{week}_weather.xls')
    df_w = data[0]
    # df = df.droplevel(0, axis=1)
    print(week, df_w.shape)

https://www.nflweather.com/en/week/2021/week-1/
<Response [200]>
week-1 (16, 13)
https://www.nflweather.com/en/week/2021/week-2/
<Response [200]>
week-2 (16, 13)
https://www.nflweather.com/en/week/2021/week-3/
<Response [200]>
week-3 (16, 13)
https://www.nflweather.com/en/week/2021/week-4/
<Response [200]>
week-4 (16, 13)
https://www.nflweather.com/en/week/2021/week-5/
<Response [200]>
week-5 (16, 13)
https://www.nflweather.com/en/week/2021/week-6/
<Response [200]>
week-6 (14, 13)
https://www.nflweather.com/en/week/2021/week-7/
<Response [200]>
week-7 (13, 13)
https://www.nflweather.com/en/week/2021/week-8/
<Response [200]>
week-8 (15, 13)
https://www.nflweather.com/en/week/2021/week-9/
<Response [200]>
week-9 (14, 13)
https://www.nflweather.com/en/week/2021/week-10/
<Response [200]>
week-10 (14, 13)
https://www.nflweather.com/en/week/2021/week-11/
<Response [200]>
week-11 (15, 13)
https://www.nflweather.com/en/week/2021/week-12/
<Response [200]>
week-12 (15, 13)
https://www.nflweather

In [41]:
data = pd.read_html('./nfl_data/weekly/weather/week-1_weather.xls')
weather_df = pd.DataFrame(data[0]) # Saves df var to dataframe
weather_df.drop(columns = ['Unnamed: 0', 'Game', 'Game.1', 'Game.2', 'Time (ET)', 'TV', 'Unnamed: 8', 'Unnamed: 12'], inplace = True) 
weather_df['Week'] = re.search(r'(\d+)', file_list[0])[0]
weather_df['Wind_Speed_MPH'] = weather_df['Wind'].map(lambda x:re.search(r'(\d+)',x)[0])
weather_df['Wind_Direction'] = weather_df['Wind'].map(lambda x:re.search(r'[A-Z]+',x)[0])
weather_df['Temp'] = weather_df['Forecast'].map(lambda x: re.search(r'(\d+)',x)[0] if x != 'DOME' else 0)
weather_df['Weather_Desc'] = weather_df['Forecast'].map(lambda x: re.search(r'\s(.+)',x)[0].strip() if x != 'DOME' else 0)
weather_df["Wind"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind"])
weather_df["Wind_Speed_MPH"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind_Speed_MPH"])
weather_df["Wind_Direction"] = np.where(weather_df["Forecast"] == "DOME", 0, weather_df["Wind_Direction"])

In [42]:
for file in file_list[1:]:
    print(file)
    data = pd.read_html(file)
    df_1 = pd.DataFrame(data[0])
    df_1.drop(columns = ['Unnamed: 0', 'Game', 'Game.1', 'Game.2', 'Time (ET)', 'TV', 'Unnamed: 8', 'Unnamed: 12'], inplace = True) 
    df_1['Week'] = re.search(r'(\d+)', file)[0]
    df_1['Wind_Speed_MPH'] = df_1['Wind'].map(lambda x:re.search(r'(\d+)',x)[0])
    df_1['Wind_Direction'] = df_1['Wind'].map(lambda x:re.search(r'[A-Z]+',x)[0])
    df_1['Temp'] = df_1['Forecast'].map(lambda x: re.search(r'(\d+)',x)[0] if x != 'DOME' else 0)
    df_1['Weather_Desc'] = df_1['Forecast'].map(lambda x: re.search(r'\s(.+)',x)[0].strip() if x != 'DOME' else 0)
    df_1["Wind"] = np.where(df_1["Forecast"] == "DOME", 0, df_1["Wind"])
    df_1["Wind_Speed_MPH"] = np.where(df_1["Forecast"] == "DOME", 0, df_1["Wind_Speed_MPH"])
    df_1["Wind_Direction"] = np.where(df_1["Forecast"] == "DOME", 0, df_1["Wind_Direction"])
    dfs = [weather_df, df_1]
    weather_df = pd.concat([weather_df.squeeze() for weather_df in dfs], ignore_index=True)

print(f'The shape of the updated df is {weather_df.shape}')

./nfl_data/weekly/weather/week-2_weather.xls
./nfl_data/weekly/weather/week-3_weather.xls
./nfl_data/weekly/weather/week-4_weather.xls
./nfl_data/weekly/weather/week-5_weather.xls
./nfl_data/weekly/weather/week-6_weather.xls
./nfl_data/weekly/weather/week-7_weather.xls
./nfl_data/weekly/weather/week-8_weather.xls
./nfl_data/weekly/weather/week-9_weather.xls
./nfl_data/weekly/weather/week-10_weather.xls
./nfl_data/weekly/weather/week-11_weather.xls
./nfl_data/weekly/weather/week-12_weather.xls
./nfl_data/weekly/weather/week-13_weather.xls
./nfl_data/weekly/weather/week-14_weather.xls
./nfl_data/weekly/weather/week-15_weather.xls
./nfl_data/weekly/weather/week-16_weather.xls
The shape of the updated df is (240, 10)


In [43]:
weather_df['Week'] = weather_df['Week'].astype(int)

## Merge `df` and `weather_df`

In [44]:
df.loc[df.Player == 'Tom Brady']

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot
81,Tom Brady,TB 31 - PHI 15,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,QB,0,0.0,0,29.0,37.0,271.0,2.0,0.0,TAM,Tampa Bay Buccaneers,Buccaneers
82,Tom Brady,LAR 30 - TB 27,15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20,QB,0,0.0,0,30.0,54.0,329.0,1.0,1.0,TAM,Tampa Bay Buccaneers,Buccaneers


In [45]:
# Change TB12 to Week 1 for upcoming merge testing
df.at[81,'Week']=1 # Change a week to ensure the join is working properly

In [46]:
df_merged = pd.merge(df, weather_df, left_on = ['Team_Name_Mascot', 'Week'], right_on = ['Home', 'Week'], how = 'inner')
df_merged

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Tom Brady,TB 31 - PHI 15,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,QB,0,0.0,0,29.0,37.0,271.0,2.0,0.0,TAM,Tampa Bay Buccaneers,Buccaneers,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,6,WSW,80,Humid and Partly Cloudy


In [50]:
df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Team_Name_Mascot'], 1, 0)
df_merged['Home_Game'] = np.where(df_merged['Home'] == df_merged['Team_Name_Mascot'], 1, 0)

In [51]:
# Test case - changes record to Away = Buccaneers, re-runs the boolean column calc. If you run this, run the next cell to return things to normal
#df_merged.at[0,'Away']='Buccaneers'
#df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Team_Name_Mascot'], 1, 0)
#df_merged

In [52]:
#df_merged.at[0,'Away']='Cowboys'
#df_merged['Away_Game'] = np.where(df_merged['Away'] == df_merged['Team_Name_Mascot'], 1, 0)
#df_merged

### Add column `opp_def_rank` to `df_merged`, to track how good of a defense this player was going against

In [53]:
Team_Abbreviations_Dict = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills' : 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GNB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KAN',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE',
    'New Orleans Saints': 'NO',
    'New York Giants': 'NYG',
    'New York Jets': 'NYJ',
    'Las Vegas Raiders': 'LVR',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'Los Angeles Chargers': 'LAC',
    'San Francisco 49ers': 'SFO',
    'Seattle Seahawks': 'SEA',
    'Los Angeles Rams': 'LAR',
    'Tampa Bay Buccaneers': 'TAM',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

In [54]:
week_num = 17

url = 'https://www.pro-football-reference.com/years/2021/opp.htm'
res = requests.get(url)
with open('./nfl_data/weekly/defensive_ranks.xls', 'wb') as f:
    f.write(res.content)

In [55]:
df_def = pd.read_html('./nfl_data/weekly/defensive_ranks.xls')
df_def = pd.DataFrame(df_def[0]) 
df_def= df_def.droplevel(0, axis=1)
df_def = df_def.rename(columns = {'Tm':'Team', 'Rk': 'Rank'})
df_def = df_def.iloc[:, [0,1]].copy() # filter to only Rank and Team
df_def['Tm_Abr'] = df_def['Team'].map(lambda x: Team_Abbreviations_Dict.get(x)) # tack on column with corresponding team abbreviation
df_def.drop(index = [32,33,34], axis = 0, inplace = True) #drop the average and total rows
df_def['Week'] = week_num
df_def['Team_Mascot'] = df_def['Team'].map(lambda x: x.split()[-1])
df_def

,Rank,Team,Tm_Abr,Week,Team_Mascot
0,1.0,Buffalo Bills,BUF,17,Bills
1,2.0,New England Patriots,NE,17,Patriots
2,3.0,Denver Broncos,DEN,17,Broncos
3,4.0,New Orleans Saints,NO,17,Saints
4,5.0,Tampa Bay Buccaneers,TAM,17,Buccaneers
5,6.0,Tennessee Titans,TEN,17,Titans
6,7.0,Dallas Cowboys,DAL,17,Cowboys
7,8.0,Kansas City Chiefs,KAN,17,Chiefs
8,9.0,San Francisco 49ers,SFO,17,49ers
9,10.0,Indianapolis Colts,IND,17,Colts


In [56]:
df_merged

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,Away_Game,Home_Game
0,Ja'Marr Chase,CIN 19 - TEN 16,15,1.0,2.0,0.0,5.0,109.0,6.0,0.0,1,WR,0,0.0,0,0.0,0.0,0.0,0.0,0.0,CIN,Cincinnati Bengals,Bengals,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,11,SW,82,Clear,0,1


In [57]:
df_merged['Opp'] = np.where(df_merged['Team_Name_Mascot'] == df_merged['Home'], df_merged['Away'], df_merged['Home'])

In [58]:
df_merged

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,Away_Game,Home_Game,Opp
0,Ja'Marr Chase,CIN 19 - TEN 16,15,1.0,2.0,0.0,5.0,109.0,6.0,0.0,1,WR,0,0.0,0,0.0,0.0,0.0,0.0,0.0,CIN,Cincinnati Bengals,Bengals,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,11,SW,82,Clear,0,1,Vikings


In [59]:
df_merged['Opp_Def_Rank'] = df_merged['Opp'].map(lambda x: df_def.loc[df_def.Team_Mascot == x]['Rank'].values[0])
df_merged

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,FGM-A Field Goals Made - Field Goals Attempted,LNG Longest field goal in terms of yards by a kicker,XPM-A Extra Points Made - Extra Points Attempted,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown,Team_Name_Abbrev,Team_Name_Full,Team_Name_Mascot,Away,Home,Forecast,Extended Forecast,Wind,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc,Away_Game,Home_Game,Opp,Opp_Def_Rank
0,Ja'Marr Chase,CIN 19 - TEN 16,15,1.0,2.0,0.0,5.0,109.0,6.0,0.0,1,WR,0,0.0,0,0.0,0.0,0.0,0.0,0.0,CIN,Cincinnati Bengals,Bengals,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,11,SW,82,Clear,0,1,Vikings,24.0


In [65]:
df_def.loc[df_def.Team_Mascot == 'Cowboys']['Rank'].values[0]

7.0